# How to understand annotations / encoded pixels ?

lets understand how to generate masks, if EncodedPixels are given instead of mask images.

In [ ]:
import numpy as np
import pandas as pd

**Shape of RGB image is (704,520,3). It means there are 704 X 520 = 3,66,080 locations.Its corresponding mask (704,520) is not given, instead they gave us EncodedPixels.**


**In the first image 0030fd0e6378 we have annotations as 118145 6 118849 7 119553 8 120257 8 120961 9 121665 10 122369 12 123074 13 123778 14 124482 15 12518... which means take 6 pixels starting from 118145 , take 7 pixels starting from 118849 , take 8 pixels starting from 119553 ....so on**

In [ ]:
df=pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv',nrows=1)

In [ ]:
df

In [ ]:
df.annotation

In [ ]:
en_pix=list(df.annotation)

In [ ]:
en_pix

This is given Encoded Pixels, lets generate a mask using it.

First lets split these numbers at space using split function:

In [ ]:
print(len(en_pix[0].split(' ')))
en_pix[0].split(' ')

Lets convert these strings into integers using map function

In [ ]:
rle = list(map(int, en_pix[0].split(' ')))
rle

First number is starting pixel and next number is count from that starting pixel. Lets bring them into 2 seperate lists

In [ ]:
pixel,pixel_count = [],[]
[pixel.append(rle[i]) if i%2==0 else pixel_count.append(rle[i]) for i in range(0, len(rle))]
print('pixel starting points:\n',pixel)
print('pixel counting:\n', pixel_count)

Lets generate masked pixel locations where exactly the mask is there using above 2 lists.

In [ ]:
rle_pixels = [list(range(pixel[i],pixel[i]+pixel_count[i])) for i in range(0, len(pixel))]
print('rle_pixels\n:', rle_pixels)

Now lets convert list of lists into a single list,

In [ ]:
rle_mask_pixels = sum(rle_pixels,[]) 
print('rle mask pixels:\n', rle_mask_pixels)

So, at these locations, input image should be masked. Lets take a black mask and make it white at these pixel locations.

Load and display the input image:

In [ ]:
import matplotlib.pyplot as plt
import cv2

In [ ]:
plt.imshow(cv2.imread('../input/sartorius-cell-instance-segmentation/train/0030fd0e6378.png'))

In [ ]:
mask_img = np.zeros((520*704,1), dtype=int)

Lets replace the above mentioned locations (rle_mask_pixels) with white pixels(255).

In [ ]:
mask_img[rle_mask_pixels] = 255

Reshape the array into input image size

In [ ]:
l,b=cv2.imread('../input/sartorius-cell-instance-segmentation/train/0030fd0e6378.png').shape[0],cv2.imread('../input/sartorius-cell-instance-segmentation/train/0030fd0e6378.png').shape[1]
mask = np.reshape(mask_img, (b, l)).T

Lets display the final mask image:

In [ ]:
plt.imshow(mask)